<a href="https://colab.research.google.com/github/MuhammedShiway/AICrowd-Mosquito/blob/main/%5B03%5D_YOLO_Det_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
data_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train_images'
train_csv_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train.csv'
yolo_train = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Train'
yolo_test = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Test'

In [3]:
# Paths
source_folder = data_path
train_folder = yolo_train
test_folder = yolo_test
csv_file = train_csv_path



---

# <center> IMPORT MODULES AND LIBRARIES </center>


---



In [4]:
import os
import random
import shutil
import csv
from sklearn.model_selection import train_test_split



---

# <center> ANNOT BBOX </center>


---



In [12]:
def annot_bbox(images_folder, csv_file, im_names):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            image_name = row['img_fName']
            if image_name in im_names:
                image_path = os.path.join(images_folder, image_name)

                txt_file_path = os.path.splitext(image_path)[0] + '.txt'

                # Extract bounding box coordinates
                img_w = int(row['img_w'])
                img_h = int(row['img_h'])
                bbx_xtl = float(row['bbx_xtl'])
                bbx_ytl = float(row['bbx_ytl'])
                bbx_xbr = float(row['bbx_xbr'])
                bbx_ybr = float(row['bbx_ybr'])

                # Calculate YOLOv8 format values
                class_label = 0  # Replace with the appropriate class label
                x_center = (bbx_xtl + bbx_xbr) / (2 * img_w)
                y_center = (bbx_ytl + bbx_ybr) / (2 * img_h)
                width = abs((bbx_xbr - bbx_xtl) / img_w)
                height = abs((bbx_ytl - bbx_ybr) / img_h)

                # Write to txt file
                with open(txt_file_path, 'w') as txt_file:
                    txt_file.write(f"{class_label} {x_center} {y_center} {width} {height}")
        print("Txt files generated successfully.")


In [13]:

# Read CSV file
with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    image_names = [row['img_fName'] for row in reader]

# Split image names into train and test sets
train_names, test_names = train_test_split(image_names, test_size=0.2, random_state=42)

In [14]:
# Generate txt files for the train folder
annot_bbox(train_folder, csv_file, train_names)

Txt files generated successfully.


In [15]:
# Generate txt files for the test folder
annot_bbox(test_folder, csv_file, test_names)

Txt files generated successfully.


In [11]:
folder_path = test_folder

# Initialize counters
image_count = 0
txt_count = 0

# List files in the folder
files = os.listdir(folder_path)

# Count images and txt files
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        if file.lower().endswith(".jpg") or file.lower().endswith(".jpeg") or file.lower().endswith(".png"):
            image_count += 1
        elif file.lower().endswith(".txt"):
            txt_count += 1

print("Number of images:", image_count)
print("Number of txt files:", txt_count)



Number of images: 1605
Number of txt files: 0


In [9]:
folder_path = test_folder

# Delete all txt files
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path) and file.lower().endswith(".txt"):
        os.remove(file_path)

print("Txt files deleted successfully.")

Txt files deleted successfully.




---

# <center> Subset 1000 </center>


---



In [16]:
destination_folder = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Train_100'
num_images_to_copy = 1000

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Get a list of all image files in the source folder
image_files = [file for file in os.listdir(train_folder) if file.lower().endswith((".jpg", ".jpeg", ".png"))]

# Shuffle the image files
random.shuffle(image_files)

# Copy the selected number of images and their corresponding txt files
for i in range(num_images_to_copy):
    image_file = image_files[i]
    image_path = os.path.join(train_folder, image_file)
    txt_file = os.path.splitext(image_file)[0] + ".txt"
    txt_path = os.path.join(train_folder, txt_file)

    # Copy the image file to the destination folder
    shutil.copy(image_path, destination_folder)

    # Copy the txt file to the destination folder
    shutil.copy(txt_path, destination_folder)

print("Images and corresponding txt files copied successfully.")

Images and corresponding txt files copied successfully.




---

# <center> YOLO V8 Training </center>


---



In [17]:
# SETUP

%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.124 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 31.2/78.2 GB disk)


In [18]:
%cd /content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Training

/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Training


In [19]:
!ls

data.yaml  runs  yolov8l.pt  yolov8n.pt  yolov8s.pt  yolov8x.pt


In [ ]:
#TRAIN THE MODEL

!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=200 imgsz=384 plots=True

Ultralytics YOLOv8.0.124 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=200, patience=50, batch=16, imgsz=384, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=Fa